In [1]:
import pandas as pd, numpy as np
from pathlib import Path
import fsspec

S3_STATIONS_TXT   = "s3://noaa-ghcn-pds/ghcnd-stations.txt"
S3_INVENTORY_TXT  = "s3://noaa-ghcn-pds/ghcnd-inventory.txt"
S3_BY_STATION     = "s3://noaa-ghcn-pds/csv/by_station/{id}.csv"
STOR = {"anon": True}

OUTDIR = Path('../data'); OUTDIR.mkdir(parents=True, exist_ok=True)
OUT_PARQUET = OUTDIR / 'ghcn_il_top4_daily.parquet'
print('Output:', OUT_PARQUET.resolve())

Output: /home/alexakt2/.jupyter/https:/data/ghcn_il_top4_daily.parquet


In [2]:
# Load Stations
colspecs = [(0,11),(12,20),(21,30),(31,37),(38,40),(41,71),(72,75),(76,79),(80,85)]
names = ['ID','LATITUDE','LONGITUDE','ELEVATION','STATE','NAME','GSN_FLAG','HCN_CRN_FLAG','WMO_ID']

stations = pd.read_fwf(S3_STATIONS_TXT, colspecs=colspecs, names=names, dtype={'ID':str,'STATE':str,'WMO_ID':str}, storage_options=STOR)
stations['NAME'] = stations['NAME'].str.strip(); stations['STATE'] = stations['STATE'].fillna('').str.strip()

inventory = pd.read_csv(
    S3_INVENTORY_TXT, sep=r'\s+', names=['ID','LAT','LON','ELEMENT','FIRSTYEAR','LASTYEAR'],
    dtype={'ID':str,'ELEMENT':str,'FIRSTYEAR':int,'LASTYEAR':int}, engine='python', storage_options=STOR
)

stations.head(), inventory.head()

(            ID  LATITUDE  LONGITUDE  ELEVATION STATE                   NAME  \
 0  ACW00011604   17.1167   -61.7833       10.1        ST JOHNS COOLIDGE FLD   
 1  ACW00011647   17.1333   -61.7833       19.2                     ST JOHNS   
 2  AE000041196   25.3330    55.5170       34.0          SHARJAH INTER. AIRP   
 3  AEM00041194   25.2550    55.3640       10.4                   DUBAI INTL   
 4  AEM00041217   24.4330    54.6510       26.8               ABU DHABI INTL   
 
   GSN_FLAG HCN_CRN_FLAG WMO_ID  
 0      NaN          NaN    NaN  
 1      NaN          NaN    NaN  
 2      GSN          NaN  41196  
 3      NaN          NaN  41194  
 4      NaN          NaN  41217  ,
             ID      LAT      LON ELEMENT  FIRSTYEAR  LASTYEAR
 0  ACW00011604  17.1167 -61.7833    TMAX       1949      1949
 1  ACW00011604  17.1167 -61.7833    TMIN       1949      1949
 2  ACW00011604  17.1167 -61.7833    PRCP       1949      1949
 3  ACW00011604  17.1167 -61.7833    SNOW       1949      194

In [3]:
# Load stations in Maine
coverage = (inventory.groupby('ID', as_index=False)
                    .agg(first=('FIRSTYEAR','min'), last=('LASTYEAR','max'))
                    .assign(years=lambda d: d['last'] - d['first'] + 1))

il = (stations.loc[stations['STATE']=='ME', ['ID','NAME','STATE','LATITUDE','LONGITUDE','ELEVATION']]
              .merge(coverage, on='ID', how='inner'))

# Updated to 1991-2020 instead of 30 years worth of data
il_timerange = il[(il['first'] <= 1991) & (il['last'] >= 2020)].copy()
il_timerange.sort_values(['years','ID'], ascending=[False, True])

,ID,NAME,STATE,LATITUDE,LONGITUDE,ELEVATION,first,last,years
411,USC00173046,GARDINER,ME,44.2203,-69.7889,40.5,1886,2025,140
347,USC00170480,BELFAST,ME,44.3953,-68.9894,36.0,1893,2025,133
405,USC00172765,FARMINGTON,ME,44.6986,-70.1619,125.3,1893,2025,133
408,USC00172878,FT KENT,ME,47.2386,-68.6119,185.9,1893,2025,133
454,USC00174927,MADISON,ME,44.7975,-69.8889,67.1,1893,2025,133
430,USC00174086,JACKMAN,ME,45.6261,-70.2461,370.0,1894,2025,132
560,USC00179891,WOODLAND,ME,45.1542,-67.3986,37.5,1917,2025,109
417,USC00173353,GREENVILLE MUNI AP,ME,45.4631,-69.5553,424.3,1920,2025,106
355,USC00170814,BRASSUA DAM,ME,45.6603,-69.8120,325.8,1929,2025,97
464,USC00175460,MOOSEHEAD,ME,45.5853,-69.7186,316.1,1930,2025,96


In [4]:
def load_station_daily(url: str) -> pd.DataFrame:
    df = pd.read_csv(url, storage_options=STOR, dtype={'ID':str,'ELEMENT':str}, parse_dates=['DATE'])
    df['DATA_VALUE'] = df['DATA_VALUE'].replace(-9999, np.nan)
    wide = (df.pivot_table(index=['ID','DATE'], columns='ELEMENT', values='DATA_VALUE', aggfunc='first').reset_index())
    for c in ('TMAX','TMIN','TAVG'):
        if c in wide: wide[c] = wide[c]/10.0
    if 'PRCP' in wide: wide['PRCP'] = wide['PRCP']/10.0
    return wide.sort_values(['ID','DATE']).reset_index(drop=True)

In [5]:
#Choosing the Rangeley, ME Station (USC00177037)

dt_daily = load_station_daily(S3_BY_STATION.format(id='USC00177037'))
dt_daily.tail()

/tmp/ipykernel_992/2833075307.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, storage_options=STOR, dtype={'ID':str,'ELEMENT':str}, parse_dates=['DATE'])


ELEMENT,ID,DATE,DAPR,DASF,MDPR,MDSF,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WT01,WT03,WT04,WT05,WT06,WT11,WT14
20484,USC00177037,2025-10-02,NaN,NaN,NaN,NaN,0.0,0.0,0.0,14.4,-2.2,-17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20485,USC00177037,2025-10-03,NaN,NaN,NaN,NaN,0.0,0.0,0.0,16.7,-2.2,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20486,USC00177037,2025-10-04,NaN,NaN,NaN,NaN,0.0,0.0,0.0,22.2,7.2,161.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20487,USC00177037,2025-10-05,NaN,NaN,NaN,NaN,0.0,0.0,0.0,24.4,4.4,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20488,USC00177037,2025-10-06,NaN,NaN,NaN,NaN,0.0,0.0,0.0,27.8,6.7,78.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Remove data that's not within 1991 and 2020

dt_daily_30yrs = dt_daily.loc[dt_daily['DATE'].between('1991-01-01', '2020-12-31')].copy()

In [7]:
#Check if there are NaNs
print(dt_daily_30yrs.isnull().sum())

#No NaNs for TMIN and TMAX! 


ELEMENT
ID          0
DATE        0
DAPR    10950
DASF    10958
MDPR    10950
MDSF    10958
PRCP       14
SNOW       23
SNWD       50
TMAX        0
TMIN        0
TOBS        0
WT01     9550
WT03    10576
WT04    10749
WT05    10922
WT06    10694
WT11    10891
WT14    10958
dtype: int64


In [ ]:
# Function that calculates the all time record high and low and the normal (mean) high and low temperature for each day

dt_daily_30yrs['DAY'] = dt_daily_30yrs['DATE'].dt.day
dt_daily_30yrs['MONTH'] = dt_daily_30yrs['DATE'].dt.month

# Record Max Temp
record_max_temp = (dt_daily_30yrs.groupby(['MONTH','DAY'])['TMAX'].max().reset_index())

# Record Min Temp
record_min_temp = (dt_daily_30yrs.groupby(['MONTH','DAY'])['TMIN'].min().reset_index())

# Average Max Temp
average_max_temp = (dt_daily_30yrs.groupby(['MONTH','DAY'])['TMAX'].mean().reset_index())

# Average Min Temp
average_min_temp = (dt_daily_30yrs.groupby(['MONTH','DAY'])['TMIN'].mean().reset_index())

# Return a pandas dataframe with the columns
dt_daily_summary = (
    record_max_temp
    .merge(record_min_temp, on=['MONTH','DAY'])
    .merge(average_min_temp, on=['MONTH','DAY'])
    .merge(average_max_temp, on=['MONTH','DAY'])
    
)
dt_daily_summary.columns = ['MONTH','DAY','record_max_temp','record_min_temp','average_min_temp','average_max_temp',]

# Print Data
print(dt_daily_summary)

     MONTH  DAY  record_max_temp  record_min_temp  average_min_temp  \
0        1    1              4.4            -27.8        -15.330000   
1        1    2              5.0            -28.9        -15.030000   
2        1    3              6.1            -28.3        -16.710000   
3        1    4              7.2            -30.6        -15.143333   
4        1    5              7.2            -32.2        -14.633333   
..     ...  ...              ...              ...               ...   
361     12   27              3.3            -29.4        -14.263333   
362     12   28              6.7            -30.0        -13.643333   
363     12   29              8.9            -29.4        -13.350000   
364     12   30              6.1            -26.7        -14.283333   
365     12   31              3.3            -29.4        -16.366667   

     average_max_temp  
0           -5.366667  
1           -4.586667  
2           -4.640000  
3           -4.540000  
4           -3.410000  
.. 

In [9]:
# Grab Data from 1996
dt_daily_1996 = dt_daily.loc[dt_daily['DATE'].between('1996-01-01', '1996-12-31')].copy()

dt_daily_1996['MONTH'] = dt_daily_1996['DATE'].dt.month
dt_daily_1996['DAY'] = dt_daily_1996['DATE'].dt.day

# Remove all other columns other than what's needed to merge
dt_daily_1996_min_max = dt_daily_1996[['MONTH','DAY','TMAX','TMIN']]

#  Merge in TMAX and TMIN from 1996
dt_daily_1996_summary = dt_daily_summary.merge(
    dt_daily_1996_min_max,
    on= ['MONTH','DAY']
    
)

dt_daily_1996_summary.rename(
    columns={'TMAX':'actual_max_temp', 'TMIN':'actual_min_temp'},
    inplace=True

)

# Print Data
print(dt_daily_1996_summary)

     MONTH  DAY  record_max_temp  record_min_temp  average_min_temp  \
0        1    1              4.4            -27.8        -15.330000   
1        1    2              5.0            -28.9        -15.030000   
2        1    3              6.1            -28.3        -16.710000   
3        1    4              7.2            -30.6        -15.143333   
4        1    5              7.2            -32.2        -14.633333   
..     ...  ...              ...              ...               ...   
361     12   27              3.3            -29.4        -14.263333   
362     12   28              6.7            -30.0        -13.643333   
363     12   29              8.9            -29.4        -13.350000   
364     12   30              6.1            -26.7        -14.283333   
365     12   31              3.3            -29.4        -16.366667   

     average_max_temp  actual_max_temp  actual_min_temp  
0           -5.366667             -2.8            -21.1  
1           -4.586667          

In [10]:
# Copied from the Bokher example

import datetime
import pandas as pd
from scipy.signal import savgol_filter

from bokeh.io import curdoc
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, DataRange1d, Select
from bokeh.palettes import Blues4
from bokeh.plotting import figure

# Columns in your DataFrame
STATISTICS = ['record_min_temp', 'actual_min_temp', 'average_min_temp', 'average_max_temp', 'actual_max_temp', 'record_max_temp']

# Use your DataFrame
df = dt_daily_1996_summary.copy()

def get_dataset(src, distribution):
    df_local = src.copy()
    df_local['date'] = pd.to_datetime(df_local['DATE']) if 'DATE' in df_local else pd.date_range(start='1996-01-01', periods=len(df_local))
    df_local['left'] = df_local['date'] - datetime.timedelta(days=0.5)
    df_local['right'] = df_local['date'] + datetime.timedelta(days=0.5)
    df_local = df_local.set_index('date')
    df_local.sort_index(inplace=True)
    
    if distribution == 'Smoothed':
        window, order = 51, 3
        for key in STATISTICS:
            df_local[key] = savgol_filter(df_local[key], window, order)

    return ColumnDataSource(data=df_local)

def make_plot(source, title):
    plot = figure(x_axis_type="datetime", width=800, tools="", toolbar_location=None)
    plot.title.text = title

    plot.quad(top='record_max_temp', bottom='record_min_temp', left='left', right='right',
              color=Blues4[2], source=source, legend_label="Record")
    plot.quad(top='average_max_temp', bottom='average_min_temp', left='left', right='right',
              color=Blues4[1], source=source, legend_label="Average")
    plot.quad(top='actual_max_temp', bottom='actual_min_temp', left='left', right='right',
              color=Blues4[0], alpha=0.5, line_color="black", source=source, legend_label="Actual")

    plot.xaxis.axis_label = None
    plot.yaxis.axis_label = "Temperature (°F)"
    plot.axis.axis_label_text_font_style = "bold"
    plot.x_range = DataRange1d(range_padding=0.0)
    plot.grid.grid_line_alpha = 0.3

    return plot

def update_plot(attrname, old, new):
    src = get_dataset(df, distribution_select.value)
    source.data.update(src.data)

# Default distribution
distribution = 'Discrete'
distribution_select = Select(value=distribution, title='Distribution', options=['Discrete', 'Smoothed'])

# Prepare source and plot
source = get_dataset(df, distribution)
plot = make_plot(source, "Daily Temperatures for Rangeley, ME Station (USC00177037) 1996")

distribution_select.on_change('value', update_plot)

controls = column(distribution_select)

curdoc().add_root(column(plot, controls))
curdoc().title = "Weather"

from bokeh.io import output_notebook, show
output_notebook()

source = get_dataset(df, distribution)
plot = make_plot(source, "Daily Temperatures for Rangeley, ME Station (USC00177037) 1996")
show(plot)




Loading BokehJS ...